<a href="https://colab.research.google.com/github/mart-anthony-stark/Modelling-and-Simulation/blob/main/Movie_Theater_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Theater Simulation

Simulation Process
- Arrive at theater
- get in line to buy ticket
- buy a ticket
- wait in line to have their ticket checked
- have their ticket checked
- decide to buy concessions or not
- buy concessions or go directly to seat


In [24]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import simpy
import random
import statistics

In [26]:
wait_times = []

## Declare Theater Class

In [27]:
class Theater(object):
  def __init__(self, env, num_cashiers, num_servers, num_ushers):
    self.env = env
    self.cashier = simpy.Resource(env, num_cashiers)
    self.server = simpy.Resource(env, num_servers)
    self.usher = simpy.Resource(env, num_ushers)

  def purchase_ticket(self, moviegoer):
    yield self.env.timeout(random.randint(1, 3)) # 1-3 mins

  def check_ticket(self, moviegoer):
    yield self.env.timeout(3 / 60) # 3 seconds

  def sell_food(self, moviegoer):
    yield self.env.timeout(random.randint(1,5)) # 1-5 mins

In [28]:
def goto_movies(env, moviegoer, theater):
  # Moviegoer arrives at the theater
  arrival_time = env.now

  with theater.cashier.request() as request:
    yield request
    yield env.process(theater.purchase_ticket(moviegoer))

  with theater.usher.request() as request:
    yield request
    yield env.process(theater.check_ticket(moviegoer))
  
  if random.choice([True, False]):
    with theater.server.request() as request:
      yield request
      yield env.process(theater.sell_food(moviegoer))

  # Moviegoer heads into the theater
  wait_times.append(env.now - arrival_time)

## Method for running the environment

In [29]:
def run_theater(env, num_cashiers, num_servers, num_ushers):
  theater = Theater(env, num_cashiers, num_servers, num_ushers)

  for moviegoer in range(3):
    env.process(goto_movies(env, moviegoer, theater))

  while True:
    yield env.timeout(0.20)  # Wait a bit before generating a new person

    moviegoer += 1
    env.process(goto_movies(env, moviegoer, theater))

In [30]:
def get_average_wait_time(wait_times):
  average_wait = statistics.mean(wait_times)
  # Pretty print the results
  minutes, frac_minutes = divmod(average_wait, 1)
  seconds = frac_minutes * 60
  return round(minutes), round(seconds)

## Method for getting the user input

In [31]:
def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. Simulation will use default values:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1, 1]
    return params

In [32]:
def main():
    # Setup
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until=90)

    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.",
    )

In [39]:
main()

Input # of cashiers working: 9
Input # of servers working: 6
Input # of ushers working: 1
Running simulation... 
The average wait time is 7 minutes and 23 seconds.
